# Transpiling a quantum circuit into Clifford+T basis

TODO: describe goal and scope of this notebook

TODO: resources for further reading

In [1]:
import sys
sys.path.append('..')

import numpy as np
import qiskit as qs

## 1. Loading QASM circuit

In [2]:
circuit_filepath = '../data/qasm_circuits/ising_n10.qasm'

circuit = qs.QuantumCircuit.from_qasm_file(circuit_filepath)
num_qubits = circuit.num_qubits

circuit.draw(fold=-1)

┌───┐ ┌──────────┐                                        ┌───┐    ┌───────────┐┌───┐┌────────────┐ ┌──────────┐                                                                                                                ┌───┐    ┌───────────┐    ┌───┐     ┌────────────┐┌──────────┐                                                                                                                   ┌───┐     ┌───────┐      ┌───┐      ┌───────┐  ┌──────────┐                                                                                                                ┌───┐     ┌──────────┐    ┌───┐    ┌───────────┐┌──────────┐                                                                                                                            ┌───┐     ┌──────────┐   ┌───┐    ┌───────────┐              ┌─┐                                                        
reg_0: ┤ H ├─┤ Rz(-0.3) ├───────────────■─────────────────■──────┤ H ├────┤ Rz(-1.92) ├┤ H ├┤ Rz(-0.288) ├─┤ Rz(-0.9) ├────────────────────────────────────────────────────────────────────────────■────────────────────────■──────────┤ H ├────┤ Rz(-0.96) ├────┤ H ├─────┤ Rz(-0.144) ├┤ Rz(-1.5) ├──────────────────────────────────────────────────────────────────────────────────■──────────────────────■─────────┤ H ├─────┤ Rz(0) ├──────┤ H ├──────┤ Rz(0) ├──┤ Rz(-2.1) ├───────────────────────────────────────────────────────────────────────────■─────────────────────────■──────────┤ H ├─────┤ Rz(0.96) ├────┤ H ├────┤ Rz(0.144) ├┤ Rz(-2.7) ├───────────────────────────────────────────────────────────────────────────────────────■─────────────────────────■──────────┤ H ├─────┤ Rz(1.92) ├───┤ H ├────┤ Rz(0.288) ├──────────────┤M├────────────────────────────────────────────────────────
       ├───┤ ├─────────┬┘ ┌─────────┐ ┌─┴─┐ ┌──────────┐┌─┴─┐ ┌──┴───┴───┐└───────────┘└───┘└────────────┘ └──────────┘    ┌───┐    ┌───────────┐┌───┐┌───────────┐  ┌─────────┐  ┌─────────┐    ┌─┴─┐     ┌──────────┐   ┌─┴─┐     ┌──┴───┴───┐└───────────┘    └───┘     └────────────┘└──────────┘    ┌───┐    ┌───────────┐    ┌───┐     ┌───────────┐ ┌─────────┐ ┌─────────┐   ┌─┴─┐   ┌──────────┐   ┌─┴─┐    ┌──┴───┴──┐  └───────┘      └───┘      └───────┘  └──────────┘  ┌───┐  ┌───────┐    ┌───┐      ┌───────┐   ┌─────────┐ ┌─────────┐     ┌─┴─┐     ┌──────────┐    ┌─┴─┐     ┌──┴───┴───┐ └──────────┘    └───┘    └───────────┘└──────────┘    ┌───┐      ┌──────────┐     ┌───┐    ┌────────────┐ ┌─────────┐  ┌─────────┐     ┌─┴─┐     ┌──────────┐    ┌─┴─┐     ┌──┴───┴───┐ └──────────┘   └───┘    └───────────┘    ┌───┐     └╥┘ ┌──────────┐     ┌───┐    ┌────────────┐         ┌─┐   
reg_1: ┤ H ├─┤ Rz(0.3) ├──┤ Rz(0.3) ├─┤ X ├─┤ Rz(-0.3) ├┤ X ├─┤ Rz(0.26) ├───────────────■──────────────────────■──────────┤ H ├────┤ Rz(-1.92) ├┤ H ├┤ Rz(0.864) ├──┤ Rz(0.9) ├──┤ Rz(0.9) ├────┤ X ├─────┤ Rz(-0.9) ├───┤ X ├─────┤ Rz(0.78) ├───────────────────■──────────────────────────■──────────┤ H ├────┤ Rz(-0.96) ├────┤ H ├─────┤ Rz(0.432) ├─┤ Rz(1.5) ├─┤ Rz(1.5) ├───┤ X ├───┤ Rz(-1.5) ├───┤ X ├────┤ Rz(1.3) ├───────────────────■────────────────────────■────────┤ H ├──┤ Rz(0) ├────┤ H ├──────┤ Rz(0) ├───┤ Rz(2.1) ├─┤ Rz(2.1) ├─────┤ X ├─────┤ Rz(-2.1) ├────┤ X ├─────┤ Rz(1.82) ├───────────────────■────────────────────────■──────────┤ H ├──────┤ Rz(0.96) ├─────┤ H ├────┤ Rz(-0.432) ├─┤ Rz(2.7) ├──┤ Rz(2.7) ├─────┤ X ├─────┤ Rz(-2.7) ├────┤ X ├─────┤ Rz(2.34) ├──────■────────────────────────■──────────┤ H ├──────╫──┤ Rz(1.92) ├─────┤ H ├────┤ Rz(-0.864) ├─────────┤M├───
       ├───┤┌┴─────────┴┐ └─────────┘ └───┘ └──────────┘└───┘┌┴──────────┤┌───────────┐┌─┴─┐ ┌──────────┐     ┌─┴─┐        ├───┤    ├───────────┤├───┤├───────────┤ ┌┴─────────┴┐ └─────────┘    └───┘     └──────────┘   └───┘    ┌┴──────────┤┌───────────┐    ┌─┴─┐      ┌──────────┐    ┌─┴─┐        ├───┤    ├───────────┤    ├───┤     ├───────────┤ ├─────────┴┐└─────────┘   └───┘   └──────────┘   └───┘    ├─────────┴┐┌──────────┐    ┌─┴─┐     ┌─────────┐    ┌─┴─┐      ├─

## 2. Transpilation using `gridsynth`

TODO: explain how to get the executables to work (need to set up proper
permissions, etc). Can go to
https://www.mathstat.dal.ca/%7Eselinger/newsynth/ and re-install them if they do
not work properly